In [1]:
from wordsegment import load, segment
load()

In [8]:
! pip install textblob
! pip install spacy
! python -m spacy download en_core_web_sm
! python -m spacy download en_core_web_md
! pip install contractions
from TextProc.Preprocessor import Preprocessor
from TextProc.preproc_configs import preproc_config_nocontext, preproc_config_context
import pandas as pd



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 19.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 10.5 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.0 MB/s eta 0:00:00a 0:00:01


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/othmaneio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [33]:
# open the text files from 'Data/twitter-datasets'
with open('Data/twitter-datasets/train_pos.txt', 'r') as f:
    train_pos = f.readlines()
with open('Data/twitter-datasets/train_neg.txt', 'r') as f:
    train_neg = f.readlines()

In [34]:
# remove \n from the end of each line
train_pos = [line[:-1] for line in train_pos]
train_neg = [line[:-1] for line in train_neg]

In [38]:
total_positive_tweets = len(train_pos)
total_negative_tweets = len(train_neg)

word = "#"  # Replace "user" with the specific word you're interested in
positive_tweets_with_user = sum(1 for tweet in train_pos if word in tweet.lower())
negative_tweets_with_user = sum(1 for tweet in train_neg if word in tweet.lower())

# Calculate probabilities
prob_positive_user = positive_tweets_with_user / (positive_tweets_with_user+negative_tweets_with_user)
prob_negative_user = negative_tweets_with_user / (positive_tweets_with_user+negative_tweets_with_user)

print(f"Probability of positive tweet given '{word}': {prob_positive_user}")
print(f"Probability of negative tweet given '{word}': {prob_negative_user}")





Probability of positive tweet given '#': 0.5893756613756613
Probability of negative tweet given '#': 0.4106243386243386


In [16]:
preprocessor = Preprocessor(preproc_config_nocontext)

In [35]:
# transform arrays to dataframes
train_pos_df = pd.DataFrame(train_pos, columns=['text'])
train_neg_df = pd.DataFrame(train_neg, columns=['text'])

# preprocess the dataframes
train_pos_df = preprocessor.preprocess(train_pos_df)
train_neg_df = preprocessor.preprocess(train_neg_df)

In [6]:
# append \n to the end of each line
train_pos_df['text'] = train_pos_df['text'].apply(lambda x: x + '\n')
train_neg_df['text'] = train_neg_df['text'].apply(lambda x: x + '\n')

In [7]:
# write the datasets to txt files
with open('Data/twitter-datasets/train_pos_preprocessed_nocontext_v1.txt', 'w') as f:
    f.writelines(train_pos_df['text'].tolist())
with open('Data/twitter-datasets/train_neg_preprocessed_nocontext_v1.txt', 'w') as f:
    f.writelines(train_neg_df['text'].tolist())

In [15]:
import contractions
import spacy
nlp = spacy.load('en_core_web_md')
import string
import re

In [16]:
def remove_punctuation(text: str) -> str:
    """
    removes all punctuation from a string
    :param text: text to remove punctuation from
    :return: text without punctuation
    """
    # replace all punctiation except ' with a space
    text = text.translate(str.maketrans(string.punctuation.replace('\'', ''), ' ' * (len(string.punctuation) - 1)))
    # remove double spaces
    text = re.sub(' +', ' ', text)
    return text


In [1]:
from TextProc.helper import correct_text, replace_hashtag, replace_emoticons, make_two_consecutive
from TextProc.Preprocessor import Preprocessor
from TextProc.preproc_configs import preproc_config_context
import pandas as pd

/opt/homebrew/Caskroom/miniconda/base/envs/tensorflow-macos/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aliessonni/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
negation_words = [
    "not", "no", "never", "none", "nobody", "nothing", "neither", "nor", "nowhere",
    "hardly", "scarcely", "barely", "rarely", "seldom", "little", "don't", "can't",
    "won't", "isn't", "aren't", "couldn't", "wasn't", "weren't", "haven't", "hasn't",
    "hadn't", "shouldn't", "wouldn't", "mustn't", "mightn't"
]

In [20]:
def word_in_tweet(tweet, words):
    return any(word in tweet.lower() for word in words)

# Calculating probabilities for target words
total_pos = len(train_pos_df)
total_neg = len(train_neg_df)
tweets_with_target_pos = train_pos_df['text'].apply(lambda tweet: word_in_tweet(tweet, negation_words)).sum()
tweets_with_target_neg = train_neg_df['text'].apply(lambda tweet: word_in_tweet(tweet, negation_words)).sum()

# Probability calculations
prob_target_pos = tweets_with_target_pos / (tweets_with_target_pos + tweets_with_target_neg)
prob_target_neg = tweets_with_target_neg / (tweets_with_target_pos + tweets_with_target_neg)

print(f"Probability of positive tweet given target word: {prob_target_pos}")
print(f"Probability of negative tweet given target word: {prob_target_neg}")

Probability of positive tweet given target word: 0.4282671281889135
Probability of negative tweet given target word: 0.5717328718110866


In [24]:
def has_unmatched_left_parentheses(text: str) -> bool:
    stack = []
    dummy = 0 

    for char in text:
        if char == '(':
            stack.append(char)
        elif char == ')':
            if len(stack) > 0:
                stack.pop()
            else:
                dummy+=1  # Unmatched right parenthesis found
        # Ignore other characters
    
    return 1 if len(stack) > 0 else 0 # Unmatched left parentheses found at the end

# Example usage:
text_example = "((Hello))))("
result = has_unmatched_left_parentheses(text_example)
print(result)

1


In [30]:
#has unmatched right parentheses
def has_unmatched_right_parentheses(text: str) -> bool:
    stack = []
    dummy = 0 

    for char in text:
        if char == '(':
            stack.append(char)
        elif char == ')':
            if len(stack) > 0:
                stack.pop()
            else:
                dummy+=1  # Unmatched right parenthesis found
        # Ignore other characters
    
    return int(dummy>0) # Unmatched left parentheses found at the end

text_example = "((Hello)()))))"
result = has_unmatched_right_parentheses(text_example)
print(result)

1


In [ ]:
# Calculating probabilities for '<user>' and '<url>'
total_tweets_pos = len(train_pos_df)
tweets_with_neg = 

In [36]:
positive_unbalanced_l_count = train_pos_df['text'].apply(lambda tweet: has_unmatched_left_parentheses(tweet)).sum()
negative_unbalanced_l_count = train_neg_df['text'].apply(lambda tweet: has_unmatched_left_parentheses(tweet)).sum()
positive_unbalanced_r_count = train_pos_df['text'].apply(lambda tweet: has_unmatched_right_parentheses(tweet)).sum()
negative_unbalanced_r_count = train_neg_df['text'].apply(lambda tweet: has_unmatched_right_parentheses(tweet)).sum()

In [37]:
p_pos_unbalanced_l = positive_unbalanced_l_count / (positive_unbalanced_l_count + negative_unbalanced_l_count)
p_neg_unbalanced_l = negative_unbalanced_l_count / (positive_unbalanced_l_count + negative_unbalanced_l_count)
p_pos_unbalanced_r = positive_unbalanced_r_count / (positive_unbalanced_r_count + negative_unbalanced_r_count)
p_neg_unbalanced_r = negative_unbalanced_r_count / (positive_unbalanced_r_count + negative_unbalanced_r_count)

print(f"Probability of positive tweet given unbalanced left parenthesis: {p_pos_unbalanced_l}")
print(f"Probability of negative tweet given unbalanced left parenthesis: {p_neg_unbalanced_l}")
print(f"Probability of positive tweet given unbalanced right parenthesis: {p_pos_unbalanced_r}")
print(f"Probability of negative tweet given unbalanced right parenthesis: {p_neg_unbalanced_r}")


Probability of positive tweet given unbalanced left parenthesis: 0.03289438031756354
Probability of negative tweet given unbalanced left parenthesis: 0.9671056196824365
Probability of positive tweet given unbalanced right parenthesis: 0.8757909962032182
Probability of negative tweet given unbalanced right parenthesis: 0.12420900379678178
